This code calculates the 14 vegetation indices (VIs) from the drone data. It reads the drone data (bands 1 to 5) for each crop considering the location and cycle, one folder at a time. After performing the calculations, it then saves the outputs to tiff files (one file for each vegetation index) inside a folder "Gr2Indices" within the output folder named as location_crop_cycle (eg. AG1_Mango_Cycle_1). The output tiffs are named as dronename_location_crop_cycle_index.tif (eg. DJIP4M_AG1_Mango_Cycle_1_DVI.tif). An 8-bit copy of all the output tiffs are also created in a folder called "COG_TIFFs_8bit" within the "Gr2Indices" folder. The naming convention of these files are as COG_dronename_location_crop_cycle_index.tif (eg. COG_DJIP4M_AG1_Mango_Cycle_1_DVI.tif). All the calculations are done locally. No temporary files are created during the calculations. 

In [ ]:
import os
import sys
import subprocess

# This gives the name of the environment directory
print("Environment name:", os.path.basename(sys.prefix))

In [ ]:
# Install necessary packages, if needed

required_packages = ["zipfile", "glob", "rasterio", "geopandas", "datetime", "re", "shutil"]

for package in required_packages:
    try:
        __import__(package if package != "scikit-learn" else "sklearn")
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("All packages have been installed!")

In [ ]:
# Import all the necessary packages
import tempfile
import os
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling
import tarfile
import geopandas as gpd
from rasterio.mask import mask
import shutil
import glob
from rasterio.crs import CRS

In [ ]:
# Folder setup
source_folder = r"C:\Users\U8019357\OneDrive - UniSQ\00_Projects\2025.06.06 A4I Crop Monitoring Vietnam\04_Data\A4I Geospatial Tech - Global Shared Folder - Put Data Here\Drone Data\Data"

# Parent folder (i.e., the folder just above "Cycle_X"):
parent_folder = r"LA2-Dragon fruit-Duong Dinh Hoi Commune, Chau Thanh District, Long An"

# Cycle
cycle = "Cycle_3"

# Output folder
output_folder = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\1 - Image Processing\Processed_Drone_VI_TIFFs"

# Enter crop name
crop_name = "Dragonfruit"

In [ ]:
input_folder = os.path.join(source_folder, parent_folder, cycle)

os.makedirs(output_folder, exist_ok=True)
print(output_folder)

# Temporary folder to extract zip files
temp_folder = tempfile.TemporaryDirectory()
print(temp_folder.name)

In [ ]:
# Extract location and crop by splitting on hyphen
parts = parent_folder.split('-')
if len(parts) < 2:
    raise ValueError(f"Expected folder format like 'AG1-Mango-...', got: {parent_folder}")

location = parts[0].strip()

# Remove spaces from crop name if present
crop = parts[1].strip().replace(' ', '')

# Create full output path
output_dir = os.path.join(output_folder, f"{location}_{crop}_{cycle}", 'Gr2Indices')
os.makedirs(output_dir, exist_ok=True)

# Add suffix for filenames
output_suffix = f"{location}_{crop}_{cycle}"

print("Output folder created at:\n", output_dir)

In [ ]:
# Initialize band mapping dictionary
band_files = {
    'Blue': None,       # Band1
    'Green': None,      # Band2
    'Red': None,        # Band3
    'RedEdge': None,    # Band4
    'NIR': None         # Band5
}

# Mapping from band number to name
band_map = {'Band1':'Blue', 'Band2':'Green', 'Band3':'Red', 'Band4':'RedEdge', 'Band5':'NIR'}

# Search for matching band files
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        for band_key, band_name in band_map.items():
            if f"_{band_key}.tif" in filename:
                band_files[band_name] = os.path.join(input_folder, filename)

# Check for missing bands
missing = [band for band, path in band_files.items() if path is None]
if missing:
    raise RuntimeError(f"❌ Missing band files: {', '.join(missing)}")

# Print found paths
print("✅ All required band files found:")
for name, path in band_files.items():
    print(f"  {name:8s}: {os.path.basename(path)}")

In [ ]:
# Paths to spectral bands from DJI Phantom 4 Multispectral drone
blue_path = band_files['Blue']      # Band1 - Blue
green_path = band_files['Green']    # Band2 - Green
red_path = band_files['Red']        # Band3 - Red
rededge_path = band_files['RedEdge'] # Band4 - Red Edge (if needed)
nir_path = band_files['NIR']        # Band5 - Near Infrared

In [ ]:
# Define output paths for selected vegetation indices using drone data
ndvi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_NDVI.tif")
evi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_EVI.tif")
savi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_SAVI.tif")
gndvi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_GNDVI.tif")
ndwi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_NDWI.tif")
cig_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_CIG.tif")
dvi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_DVI.tif")
rvi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_RVI.tif")
tvi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_TVI.tif")
msavi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_MSAVI.tif")
pri_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_PRI.tif")
wdrvi_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_WDRVI.tif")
vari_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_VARI.tif")
ari_path = os.path.join(output_dir, f"DJIP4M_{location}_{crop}_{cycle}_ARI.tif")

In [ ]:
# 1. Calculating NDVI, EVI, SAVI, GNDVI

with rasterio.open(nir_path) as nir_src, \
     rasterio.open(red_path) as red_src, \
     rasterio.open(green_path) as green_src, \
     rasterio.open(blue_path) as blue_src:

    NODATA_VALUE = np.float32(-32767.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    ndvi_path = os.path.join(output_dir, f"DJIP4M_{output_suffix}_NDVI.tif")
    evi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_EVI.tif")
    savi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_SAVI.tif")
    gndvi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_GNDVI.tif")

    with rasterio.open(ndvi_path, 'w', **meta) as ndvi_dst, \
         rasterio.open(evi_path, 'w', **meta) as evi_dst, \
         rasterio.open(savi_path, 'w', **meta) as savi_dst, \
         rasterio.open(gndvi_path, 'w', **meta) as gndvi_dst:

        nir_nodata = nir_src.nodata or 0
        red_nodata = red_src.nodata or 0
        green_nodata = green_src.nodata or 0
        blue_nodata = blue_src.nodata or 0

        for idx, window in nir_src.block_windows(1):
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)

            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale reflectance values from integer DN to [0,1]
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE

            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)

            with np.errstate(divide='ignore', invalid='ignore'):
                ndvi = (nir_masked - red_masked) / (nir_masked + red_masked)
                ndvi = np.where(np.isfinite(ndvi) & (ndvi >= -1) & (ndvi <= 1), ndvi, NODATA_VALUE)

                evi = 2.5 * (nir_masked - red_masked) / (nir_masked + 6 * red_masked - 7.5 * blue_masked + 1)
                evi = np.where(np.isfinite(evi) & (evi >= -1) & (evi <= 1), evi, NODATA_VALUE)

                L = 0.5
                savi_denom = nir + red + L
                savi = np.where(valid_mask & (savi_denom != 0), ((nir - red) * (1 + L)) / savi_denom, NODATA_VALUE)
                savi = np.where((savi >= -0.3) & (savi <= 1.0), savi, NODATA_VALUE)

                gndvi_denom = nir + green
                gndvi = np.where(valid_mask & (gndvi_denom != 0), (nir - green) / gndvi_denom, NODATA_VALUE)
                gndvi = np.where((gndvi >= -1.0) & (gndvi <= 1.0), gndvi, NODATA_VALUE)

            ndvi_dst.write(ndvi, 1, window=window)
            evi_dst.write(evi, 1, window=window)
            savi_dst.write(savi, 1, window=window)
            gndvi_dst.write(gndvi, 1, window=window)

In [ ]:
# 1. Save final outputs for NDVI, EVI, SAVI, GNDVI
index_files = [(ndvi_path, "NDVI"), (evi_path, "EVI"), (savi_path, "SAVI"), (gndvi_path, "GNDVI")]

for filepath, index_name in index_files:
    try:
        with rasterio.open(filepath, 'r') as src:
            meta = src.meta.copy()
            data = src.read(1)
            nodata = src.nodata if src.nodata is not None else -9999

        meta.update(compress='lzw')

        # Write to temporary file in same directory
        temp_path = filepath + ".temp"
        with rasterio.open(temp_path, 'w', **meta) as dst:
            dst.write(data, 1)

        # Replace original file
        shutil.move(temp_path, filepath)
        print(f" {index_name} successfully saved with compression to: {os.path.basename(filepath)}")

    except Exception as e:
        print(f" Error saving {index_name}: {e}")

In [ ]:
# 2. Calculating NDWI, CIG, DVI, TVI

with rasterio.open(nir_path) as nir_src, \
     rasterio.open(red_path) as red_src, \
     rasterio.open(green_path) as green_src, \
     rasterio.open(blue_path) as blue_src:

    NODATA_VALUE = np.float32(-32767.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    # Define direct output paths (adjust names as needed)
    ndwi_path = os.path.join(output_dir, f"DJIP4M_{output_suffix}_NDWI.tif")
    cig_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_CIG.tif")
    dvi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_DVI.tif")
    tvi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_TVI.tif")

    with rasterio.open(ndwi_path, 'w', **meta) as ndwi_dst, \
         rasterio.open(cig_path, 'w', **meta) as cig_dst, \
         rasterio.open(dvi_path, 'w', **meta) as dvi_dst, \
         rasterio.open(tvi_path, 'w', **meta) as tvi_dst:

        nir_nodata = nir_src.nodata or 0
        red_nodata = red_src.nodata or 0
        green_nodata = green_src.nodata or 0
        blue_nodata = blue_src.nodata or 0

        for idx, window in nir_src.block_windows(1):
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)

            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE

            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)

            with np.errstate(divide='ignore', invalid='ignore'):
                ndwi_denom = green_masked + nir_masked
                ndwi = np.where(valid_mask & (ndwi_denom != 0),
                                (green_masked - nir_masked) / ndwi_denom, NODATA_VALUE)
                ndwi = np.where((ndwi >= -1.0) & (ndwi <= 1.0), ndwi, NODATA_VALUE)

                cig = np.where(valid_mask & (green_masked != 0),
                               nir_masked / green_masked - 1, NODATA_VALUE)
                cig = np.where((cig >= 0.0) & (cig <= 5.5), cig, NODATA_VALUE)

                dvi = np.where(valid_mask, nir_masked - red_masked, NODATA_VALUE)
                dvi = np.where((dvi >= -1.0) & (dvi <= 1.0), dvi, NODATA_VALUE)

                ndvi_denom = nir_masked + red_masked
                ndvi = np.where(ndvi_denom != 0,
                                (nir_masked - red_masked) / ndvi_denom, NODATA_VALUE)
                ndvi = np.where((ndvi >= -1.0) & (ndvi <= 1.0), ndvi, NODATA_VALUE)
                tvi_raw = np.sqrt(np.clip(ndvi + 0.5, a_min=0, a_max=None))
                tvi = np.where((tvi_raw > 0.0) & (tvi_raw <= 1.3), tvi_raw, NODATA_VALUE)

            # Replace nan with nodata
            ndwi = np.nan_to_num(ndwi, nan=NODATA_VALUE).astype(np.float32)
            cig = np.nan_to_num(cig, nan=NODATA_VALUE).astype(np.float32)
            dvi = np.nan_to_num(dvi, nan=NODATA_VALUE).astype(np.float32)
            tvi = np.nan_to_num(tvi, nan=NODATA_VALUE).astype(np.float32)

            ndwi_dst.write(ndwi, 1, window=window)
            cig_dst.write(cig, 1, window=window)
            dvi_dst.write(dvi, 1, window=window)
            tvi_dst.write(tvi, 1, window=window)

In [ ]:
# 2. Save final outputs for NDWI, CIG, DVI, TVI
index_files = [(ndwi_path, "NDWI"), (cig_path, "CIG"), (dvi_path, "DVI"), (tvi_path, "TVI")]

for filepath, index_name in index_files:
    try:
        with rasterio.open(filepath, 'r') as src:
            meta = src.meta.copy()
            data = src.read(1)
            nodata = src.nodata if src.nodata is not None else -9999

        meta.update(compress='lzw')

        # Write to temporary file in same directory
        temp_path = filepath + ".temp"
        with rasterio.open(temp_path, 'w', **meta) as dst:
            dst.write(data, 1)

        # Replace original file
        shutil.move(temp_path, filepath)

        print(f" {index_name} successfully saved with compression to: {os.path.basename(filepath)}")

    except Exception as e:
        print(f" Error saving {index_name}: {e}")

In [ ]:
# 3. Calculating RVI, MSAVI, PRI, WDRVI
with rasterio.open(nir_path) as nir_src, \
     rasterio.open(red_path) as red_src, \
     rasterio.open(green_path) as green_src, \
     rasterio.open(blue_path) as blue_src:

    # Define a NoData value
    NODATA_VALUE = np.float32(-32767.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    rvi_path = os.path.join(output_dir, f"DJIP4M_{output_suffix}_RVI.tif")
    msavi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_MSAVI.tif")
    pri_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_PRI.tif")
    wdrvi_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_WDRVI.tif")

    # Get nodata values from each band or default to 0
    nir_nodata = nir_src.nodata or 0
    red_nodata = red_src.nodata or 0
    green_nodata = green_src.nodata or 0
    blue_nodata = blue_src.nodata or 0

    with rasterio.open(rvi_path, 'w', **meta) as rvi_dst, \
         rasterio.open(msavi_path, 'w', **meta) as msavi_dst, \
         rasterio.open(pri_path, 'w', **meta) as pri_dst, \
         rasterio.open(wdrvi_path, 'w', **meta) as wdrvi_dst:
         
        for idx, window in nir_src.block_windows(1):
            # Read raw data
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)
            
            # Create unified invalid mask: NoData or zero in any band
            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale and mask invalid pixels
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

            # Apply mask
            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE
        
            # Create valid mask for index calculations
            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)

            with np.errstate(divide='ignore', invalid='ignore'):

                # Ratio Vegetation Index (RVI)
                rvi = np.where(red_masked != 0, nir_masked / red_masked, NODATA_VALUE)
                rvi = np.where((rvi >= 0.5) & (rvi <= 10.0), rvi, NODATA_VALUE)

                # Modified Soil-Adjusted Vegetation Index (MSAVI)
                msavi_inner = (2 * nir_masked + 1) ** 2 - 8 * (nir_masked - red_masked)
                msavi = ((2 * nir_masked + 1) - np.sqrt(np.clip(msavi_inner, a_min=0, a_max=None))) / 2
                msavi = np.where((msavi >= -1.0) & (msavi <= 1.0), msavi, NODATA_VALUE)

                # Photochemical Reflectance Index (PRI)
                pri_denom = green_masked + red_masked
                pri = np.where(pri_denom != 0, (green_masked - red_masked) / pri_denom, NODATA_VALUE)
                pri = np.where((pri >= -1.0) & (pri <= 1.0), pri, NODATA_VALUE)

                # Wide Dynamic Range Vegetation Index (WDRVI)
                wdrvi_denom = 0.1 * nir_masked + red_masked
                wdrvi = np.where(wdrvi_denom != 0, (0.1 * nir_masked - red_masked) / wdrvi_denom, NODATA_VALUE)
                wdrvi = np.where((wdrvi >= -1.0) & (wdrvi <= 1.0), wdrvi, NODATA_VALUE)

            # Replace np.nan with NODATA_VALUE for additional indices
            rvi = np.nan_to_num(rvi, nan=NODATA_VALUE).astype(np.float32)
            msavi = np.nan_to_num(msavi, nan=NODATA_VALUE).astype(np.float32) 
            pri = np.nan_to_num(pri, nan=NODATA_VALUE).astype(np.float32) 
            wdrvi = np.nan_to_num(wdrvi, nan=NODATA_VALUE).astype(np.float32) 
                        
            # Write each index to disk
            rvi_dst.write(rvi, 1, window=window)
            msavi_dst.write(msavi, 1, window=window)
            pri_dst.write(pri, 1, window=window)
            wdrvi_dst.write(wdrvi, 1, window=window)    

In [ ]:
# 3. Save final outputs for RVI, MSAVI, PRI, WDRVI
index_files = [
    (ndwi_path, "NDWI"),
    (cig_path, "CIG"),
    (dvi_path, "DVI"),
    (tvi_path, "TVI")
]

for filepath, index_name in index_files:
    try:
        with rasterio.open(filepath, 'r') as src:
            meta = src.meta.copy()
            data = src.read(1)
            nodata = src.nodata if src.nodata is not None else -9999

        meta.update(compress='lzw')

        # Write to temporary file in same directory
        temp_path = filepath + ".temp"
        with rasterio.open(temp_path, 'w', **meta) as dst:
            dst.write(data, 1)

        # Replace original file
        shutil.move(temp_path, filepath)

        print(f" {index_name} successfully saved with compression to: {os.path.basename(filepath)}")

    except Exception as e:
        print(f" Error saving {index_name}: {e}")

In [ ]:
# 4. Calculating VARI, ARI
with rasterio.open(nir_path) as nir_src, \
     rasterio.open(red_path) as red_src, \
     rasterio.open(green_path) as green_src, \
     rasterio.open(blue_path) as blue_src:

    # Define a NoData value
    NODATA_VALUE = np.float32(-32767.0)
    meta = nir_src.meta.copy()
    meta.update(dtype='float32', count=1, compress='lzw', nodata=NODATA_VALUE)

    vari_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_VARI.tif")
    ari_path  = os.path.join(output_dir, f"DJIP4M_{output_suffix}_ARI.tif")

    # Get nodata values or default to 0
    nir_nodata = nir_src.nodata or 0
    red_nodata = red_src.nodata or 0
    green_nodata = green_src.nodata or 0
    blue_nodata = blue_src.nodata or 0

    with rasterio.open(vari_path, 'w', **meta) as vari_dst, \
         rasterio.open(ari_path, 'w', **meta) as ari_dst:

        for idx, window in nir_src.block_windows(1):
            # Read raw data
            nir_raw = nir_src.read(1, window=window)
            red_raw = red_src.read(1, window=window)
            green_raw = green_src.read(1, window=window)
            blue_raw = blue_src.read(1, window=window)

            # Create unified invalid mask: NoData or zero in any band
            invalid_mask = (
                (nir_raw == 0) | (nir_raw == nir_nodata) |
                (red_raw == 0) | (red_raw == red_nodata) |
                (green_raw == 0) | (green_raw == green_nodata) |
                (blue_raw == 0) | (blue_raw == blue_nodata)
            )

            # Scale and mask invalid pixels
            nir = nir_raw.astype(np.float32) * 0.0001
            red = red_raw.astype(np.float32) * 0.0001
            green = green_raw.astype(np.float32) * 0.0001
            blue = blue_raw.astype(np.float32) * 0.0001

            # Apply mask
            nir[invalid_mask] = NODATA_VALUE
            red[invalid_mask] = NODATA_VALUE
            green[invalid_mask] = NODATA_VALUE
            blue[invalid_mask] = NODATA_VALUE
        
            # Create valid mask for index calculations
            valid_mask = ~invalid_mask

            nir_masked = np.where(valid_mask, nir, np.nan)
            red_masked = np.where(valid_mask, red, np.nan)
            blue_masked = np.where(valid_mask, blue, np.nan)
            green_masked = np.where(valid_mask, green, np.nan)

            with np.errstate(divide='ignore', invalid='ignore'):
                # Visible Atmospherically Resistant Index (VARI)
                vari_denom = green_masked + red_masked - blue_masked
                vari = np.where(vari_denom != 0, (green_masked - red_masked) / vari_denom, NODATA_VALUE)
                vari = np.where((vari >= -1.0) & (vari <= 1.0), vari, NODATA_VALUE)

                # Anthocyanin Reflectance Index (ARI)
                ari = np.where((green_masked != 0) & (red_masked != 0), (1 / green_masked) - (1 / red_masked), NODATA_VALUE)
                ari = np.where((ari >= -6.0) & (ari <= 6.0), ari, NODATA_VALUE)

            # Replace np.nan with NODATA_VALUE for additional indices
            vari = np.nan_to_num(vari, nan=NODATA_VALUE).astype(np.float32)
            ari = np.nan_to_num(ari, nan=NODATA_VALUE).astype(np.float32)
                       
            # Write each index to disk
            vari_dst.write(vari, 1, window=window)
            ari_dst.write(ari, 1, window=window)

In [ ]:
# 4. Save final outputs for VARI and ARI
index_files = [(vari_path, "VARI"), (ari_path, "ARI")]

for filepath, index_name in index_files:
    try:
        with rasterio.open(filepath, 'r') as src:
            meta = src.meta.copy()
            data = src.read(1)
            nodata = src.nodata if src.nodata is not None else -9999

        meta.update(compress='lzw')

        # Write to temporary file in same directory
        temp_path = filepath + ".temp"
        with rasterio.open(temp_path, 'w', **meta) as dst:
            dst.write(data, 1)

        # Replace original file
        shutil.move(temp_path, filepath)

        print(f" {index_name} successfully saved with compression to: {os.path.basename(filepath)}")

    except Exception as e:
        print(f" Error saving {index_name}: {e}")

In [ ]:
# Converting GeoTiffs to Cloud Optimized GeoTIFFs (COGs) : Open source and widely used

source_folder = output_dir
output_folder_GOC = os.path.join(source_folder, "COG_TIFFs_8bit")
os.makedirs(output_folder_GOC, exist_ok=True)

# Find all .TIF files
tiff_files = glob.glob(os.path.join(source_folder, "*.TIF"))

for tiff_path in tiff_files:
    filename = os.path.basename(tiff_path)
    output_filename = "COG_" + filename
    output_path = os.path.join(output_folder_GOC, output_filename)


    with rasterio.open(tiff_path) as src:
        data = src.read(1)  
        profile = src.profile.copy()
        nodata_in = -9999.0
        nodata_out = 255

        # Mask NoData values
        if nodata_in is not None:
            data = np.where(data == nodata_in, np.nan, data)

        # Rescale to 0–254
        min_val = np.nanmin(data)
        max_val = np.nanmax(data)

        if max_val == min_val:
            scaled = np.zeros_like(data)
        else:
            scaled = 254 * (data - min_val) / (max_val - min_val)

        # Set nodata areas to 255
        scaled_data = np.where(np.isnan(scaled), nodata_out, scaled).astype(np.uint8)

        # Update profile for 8-bit COG
        profile.update({
            "driver": "COG",
            "dtype": "uint8",
            "count": 1,
            "compress": "DEFLATE",
            "blocksize": 512,
            "tiled": True,
            "BIGTIFF": "IF_SAFER",
            "nodata": nodata_out
        })

        # Write the output
        with rasterio.open(output_path, "w", **profile) as dst:
            dst.write(scaled_data, 1)

In [ ]:
# Revert the band files to cloud-only
for bfile in [blue_path, green_path, red_path, rededge_path, nir_path]:
    print(f"Cleaning {os.path.basename(bfile)} ...")
    subprocess.run(["attrib", "+U", "-P", bfile], check=True)